In [12]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
from datetime import datetime
from math import isnan

import threading
import time
import pandas
from pandas import DataFrame

In [13]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []  # Initialize variable to store candle
        self.nextorderId = None

    def historicalData(self, reqId, bar):
        print(f'reqId: {reqId} Time: {bar.date} Close: {bar.close}')
        self.data.append([reqId, bar.date, bar.close])

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)

    def orderStatus(self, orderId, status, filled, remaining, avgFullPrice, permId, parentId, lastFillPrice, clientId,
                    whyHeld, mktCapPrice):
        print('orderStatus - orderid:', orderId, 'status:', status, 'filled', filled, 'remaining', remaining,
              'lastFillPrice', lastFillPrice)

    def openOrder(self, orderId, contract, order, orderState):
        print('openOrder id:', orderId, contract.symbol, contract.secType, '@', contract.exchange, ':', order.action,
              order.orderType, order.totalQuantity, orderState.status)

    def execDetails(self, reqId, contract, execution):
        print('Order Executed: ', reqId, contract.symbol, contract.secType, contract.currency, execution.execId,
              execution.orderId, execution.shares, execution.lastLiquidity)
    # def updateAccountValue(self, key:str, val:str, currency:str,
    #                         accountName:str):
    def accountSummary(self, reqId:int, account:str, tag:str, value:str,
                       currency:str):
        print(f'Account summary: account[{account}], value[{value}], currency[{currency}]')


In [14]:
app = IBapi()
app.connect('127.0.0.1', 7497, 10645)


def run_loop():
    app.run()

In [15]:
reqId_serial = 1

In [16]:
# Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()
time.sleep(1)  # Sleep interval to allow time for connection to server

ERROR -1 2104 Market data farm connection is OK:cafarm
ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:eufarmnj
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:uscrypto
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:afarm
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:cashhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


The next valid order id is:  1


In [17]:
def create_contract(curr1, curr2):
    # Create contract object
    contract = Contract()
    contract.symbol = curr1
    contract.secType = 'CASH'
    contract.exchange = 'IDEALPRO'
    contract.currency = curr2
    return contract

In [18]:
EUR_USD_contract = create_contract('EUR', 'USD')

In [19]:
# Request historical candles
app.reqHistoricalData(reqId=1, contract=EUR_USD_contract, endDateTime='', durationStr='1 D', barSizeSetting='1 day',
                      whatToShow='MIDPOINT', useRTH=0, formatDate=1, keepUpToDate=False, chartOptions=[])
time.sleep(2)  # sleep to allow enough time for data to be returned

reqId: 1 Time: 20220422 Close: 1.07975


In [20]:
df = pandas.DataFrame(app.data, columns=['reqId', 'DateTime', 'Close'])

In [21]:
df['DateTime'] = [datetime.strptime(dt, '%Y%m%d') for dt in df['DateTime']]

In [22]:
df

,reqId,DateTime,Close
0,1,2022-04-22,1.07975


In [23]:
def request_data(contract):
    global reqId_serial
    reqId_serial += 1
    app.reqHistoricalData(reqId=reqId_serial, contract=contract, endDateTime='', durationStr='1 D',
                          barSizeSetting='1 day',
                          whatToShow='MIDPOINT', useRTH=0, formatDate=1, keepUpToDate=False, chartOptions=[])
    time.sleep(0.5)
    if app.data[len(app.data) - 1][0] != reqId_serial:
        return float('NaN')
    else:
        return app.data[len(app.data) - 1][2]

In [24]:
print(request_data(EUR_USD_contract))

reqId: 2 Time: 20220422 Close: 1.07975
1.07975


In [25]:
def fetch_exc_rate(base, quote):
    contract = create_contract(base, quote)
    return request_data(contract)

In [26]:
currencies = ['USD', 'EUR', 'GBP', 'JPY', 'AUD']

In [27]:
def fetch_all(curr_list):
    matrix = DataFrame(columns=curr_list, index=curr_list)
    for i in range(0, len(curr_list)):
        for j in range(0, len(curr_list)):
            if i == j:
                matrix[curr_list[i]][curr_list[j]] = 1
            else:
                matrix[curr_list[i]][curr_list[j]] = fetch_exc_rate(curr_list[i], curr_list[j])
    return matrix

In [28]:
exchange_rates = fetch_all(currencies)

ERROR 3 200 No security definition has been found for the request
ERROR 4 200 No security definition has been found for the request


reqId: 5 Time: 20220422 Close: 128.4825


ERROR 6 200 No security definition has been found for the request


reqId: 7 Time: 20220422 Close: 1.07975
reqId: 8 Time: 20220422 Close: 0.84125
reqId: 9 Time: 20220422 Close: 138.771
reqId: 10 Time: 20220422 Close: 1.49105


ERROR 12 200 No security definition has been found for the request
ERROR 15 200 No security definition has been found for the request
ERROR 16 200 No security definition has been found for the request
ERROR 17 200 No security definition has been found for the request
ERROR 18 200 No security definition has been found for the request


reqId: 11 Time: 20220422 Close: 1.28395
reqId: 13 Time: 20220422 Close: 165.0055
reqId: 14 Time: 20220422 Close: 1.77185
reqId: 19 Time: 20220422 Close: 0.724


ERROR 20 200 No security definition has been found for the request
ERROR 21 200 No security definition has been found for the request


reqId: 22 Time: 20220422 Close: 93.0875


In [19]:
def fill_in_nan(matrix):
    for col in matrix.columns:
        for row in matrix.index:
            if isnan(matrix[col][row]) & (not isnan(matrix[row][col])):
                matrix[col][row] = 1 / matrix[row][col]
    return matrix

In [59]:
def check_all_data(matrix):
    return not matrix.isnull().values.any()


In [57]:
exchange_rates = fill_in_nan(exchange_rates)

In [60]:
check_all_data(exchange_rates)

True

In [61]:
exchange_rates

,USD,EUR,GBP,JPY,AUD
USD,1,1.07875,1.2827,0.007777,0.72385
EUR,0.926999,1,1.189061,0.007209,0.670961
GBP,0.779606,0.841,1,0.006063,0.564318
JPY,128.5795,138.7225,164.9465,1,93.076
AUD,1.381502,1.4904,1.77205,0.010744,1


In [62]:
def check_arbitrage(result_dict, matrix, curr1, curr2, curr3):
    arbitrage_amount = matrix[curr1][curr2] * matrix[curr2][curr3] * matrix[curr3][curr1] - 1
    result_dict[curr1 + '.' + curr2 + '.' + curr3] = arbitrage_amount
    arbitrage_amount_rev = matrix[curr1][curr3] * matrix[curr3][curr2] * matrix[curr2][curr1] - 1
    result_dict[curr1 + '.' + curr3 + '.' + curr2] = arbitrage_amount_rev
    print(curr1 + '->' + curr2 + '->' + curr3 + '->' + curr1 + ': ' + "{:.6f}".format(arbitrage_amount))
    print(curr1 + '->' + curr3 + '->' + curr2 + '->' + curr1 + ': ' + "{:.6f}".format(arbitrage_amount_rev))
    return arbitrage_amount

In [63]:
def check_all_arbitrage(result_dict, matrix, currency_list):
    for i in range(0, len(currency_list)):
        for j in range(i + 1, len(currency_list)):
            for k in range(j + 1, len(currency_list)):
                check_arbitrage(result_dict, matrix, currency_list[i], currency_list[j], currency_list[k])
    return result_dict

In [65]:
results = dict()
results = check_all_arbitrage(results, exchange_rates, currencies)

USD->EUR->GBP->USD: 0.000001
USD->GBP->EUR->USD: -0.000001
USD->EUR->JPY->USD: 0.000125
USD->JPY->EUR->USD: -0.000125
USD->EUR->AUD->USD: 0.000070
USD->AUD->EUR->USD: -0.000070
USD->GBP->JPY->USD: 0.000107
USD->JPY->GBP->USD: -0.000107
USD->GBP->AUD->USD: -0.000001
USD->AUD->GBP->USD: 0.000001
USD->JPY->AUD->USD: -0.000040
USD->AUD->JPY->USD: 0.000040
EUR->GBP->JPY->EUR: -0.000018
EUR->JPY->GBP->EUR: 0.000018
EUR->GBP->AUD->EUR: -0.000071
EUR->AUD->GBP->EUR: 0.000071
EUR->JPY->AUD->EUR: 0.000015
EUR->AUD->JPY->EUR: -0.000015
GBP->JPY->AUD->GBP: 0.000068
GBP->AUD->JPY->GBP: -0.000068


In [67]:
max_path = max(results, key=results.get)

'USD.EUR.JPY'

In [20]:
def FX_order_contract(curr1, curr2):
	contract = Contract()
	contract.symbol = curr1
	contract.secType = 'CASH'
	contract.exchange = 'IDEALPRO'
	contract.currency = curr2
	return contract


In [22]:
def create_order(quantity, action='BUY', orderType='MKT', lmtPrice=None):
    order = Order()
    order.action = action
    order.totalQuantity = quantity
    order.orderType = orderType
    if orderType == 'LMT' and lmtPrice is not None:
        order.lmtPrice = lmtPrice
    return order

In [28]:
usd_eur_order = create_order(100000,'BUY','MKT')
usd_eur_contract = FX_order_contract('USD','EUR')
app.placeOrder(app.nextValidId, usd_eur_contract, usd_eur_order)

AttributeError: 'NoneType' object has no attribute 'action'

In [30]:
order = Order
order.action = 'BUY'
order.totalQuantity =100000
order.orderType =  'MKT'
order.lmtPrice = 1.10

app.placeOrder(app.nextValidId, usd_eur_contract, order)

AttributeError: type object 'Order' has no attribute 'auxPrice'